In [1]:
import torch
import torch.nn.functional as F
torch.set_printoptions(precision=10)

def make_tensor(s, requires_grad=False, dtype="torch.float32"):
    return eval(f"torch.tensor({s.strip().replace('{', '[').replace('}', ']')}, requires_grad={requires_grad}, dtype={dtype})")

def from_tensor(x):
    return str(x)[7:-1].replace("[", "{").replace("]", "}")

In [2]:
l = torch.nn.Linear(2, 3)

l.weight = torch.nn.Parameter(make_tensor('''{{ 0.284576,  0.270818, -0.960440},
                                              {-1.163150,  0.014820, -0.673953}}''', True))
l.weight.register_hook(lambda grad: print("l.weight grad: ", grad))

l.bias = torch.nn.Parameter(make_tensor('{{-0.365125,  0.889674}}', True))
l.bias.register_hook(lambda grad: print("l.bias grad: ", grad))

In [3]:
model = torch.nn.Sequential(
    l,
    torch.nn.Sigmoid()
)

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [5]:
x = make_tensor("{{1.0, 2.5, 7.3}}", True)
x.register_hook(lambda grad: print("x_grad: ", grad))

In [6]:
y = model(x)
y

tensor([[0.0016346080, 0.0057300595]], grad_fn=<SigmoidBackward>)

In [7]:
z = make_tensor("{{0, 1}}")

In [8]:
loss = torch.nn.MSELoss()(y, z)
loss.item()

0.49428772926330566

In [9]:
loss.backward()

l.bias grad:  tensor([[ 2.6675756999e-06, -5.6645805016e-03]])
x_grad:  tensor([[ 6.5895156004e-03, -8.3226652350e-05,  3.8150988985e-03]])
l.weight grad:  tensor([[ 2.6675756999e-06,  6.6689390223e-06,  1.9473303837e-05],
        [-5.6645805016e-03, -1.4161450788e-02, -4.1351437569e-02]])


In [10]:
optimizer.step()

In [11]:
l.weight

Parameter containing:
tensor([[ 0.2845759690,  0.2708179355, -0.9604401588],
        [-1.1630933285,  0.0149616143, -0.6735394597]], requires_grad=True)

In [12]:
l.bias

Parameter containing:
tensor([[-0.3651250303,  0.8897306323]], requires_grad=True)

In [15]:
print(from_tensor(torch.tensor([[-0.365125,  0.889731]], requires_grad=True)))

{{-0.3651250005,  0.8897309899}}, requires_grad=True
